## _*Particle hole transformation of FermionicOperator*_

This notebook demonstrates carrying out a ParticleHole transformation on the FermionicOperator in Qiskit Chemistry. Here we use the FermionicOperator directly to demonstrate.

Note: The Hamiltonian class that wraps this provides a means to use either full, or particle hole transformation. Under the covers it does what is shown here though.

This notebook has been written to use the PYSCF chemistry driver.

In [1]:
import numpy as np

from qiskit import BasicAer

from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.aqua.components.optimizers import L_BFGS_B
from qiskit.circuit.library import TwoLocal

from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType

We'll do this with H2 molecule and use the PySCF driver to create the integrals we need for the FermionicOperator.

In [2]:
driver = PySCFDriver(atom='H .0 .0 .0; H .0 .0 0.735', unit=UnitsType.ANGSTROM,
                     charge=0, spin=0, basis='sto3g')
molecule = driver.run()

We first create the FermionicOperator and use NumPyMinimumEigensolver with qubit operator we get from it via a jordan wigner mapping to compute the ground state energy. Here this is the electronic component of the total ground state energy (the total ground state energy would include the nuclear repulsion energy we can get from the molecule that comes from the driver)

In [3]:
ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
qubitOp_jw = ferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
qubitOp_jw.chop(10**-10)

# Using NumPyMinimumEigensolver to get the smallest eigenvalue
exact_eigensolver = NumPyMinimumEigensolver(qubitOp_jw)
ret = exact_eigensolver.run()

print('The exact ground state energy is: {}'.format(ret.eigenvalue.real))
print('The Hartree Fock Electron Energy is: {}'.format(molecule.hf_energy - molecule.nuclear_repulsion_energy))

The exact ground state energy is: -1.8572750302023824
The Hartree Fock Electron Energy is: -1.8369679912029842


Now the same as above but with ParticleHole transformation. This removes out energy from the FermionicOperator that is equivalent to the electronic part of the Hartree Fock Energy that we also computed above. The Hartree Fock energy also comes from the driver. To get the total electronic ground state energy we need to add the part we now compute with the part that was removed by the transformation.

In [4]:
# particle hole transformation
newferOp, energy_shift = ferOp.particle_hole_transformation([molecule.num_alpha, molecule.num_beta])
print('Energy shift is: {}'.format(energy_shift))
newqubitOp_jw = newferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
newqubitOp_jw.chop(10**-10)

exact_eigensolver = NumPyMinimumEigensolver(newqubitOp_jw)
ret = exact_eigensolver.run()

print('The exact ground state energy in PH basis is {}'.format(ret.eigenvalue.real))
print('The exact ground state energy in PH basis is {} (with energy_shift)'.format(ret.eigenvalue.real - energy_shift))

Energy shift is: 1.8369679912029837
The exact ground state energy in PH basis is -0.020307038999395295
The exact ground state energy in PH basis is -1.857275030202379 (with energy_shift)


We run here using the quantum VQE algorithm to show the same result. The parameters printed are the optimal parameters of the variational form at the minimum energy, the ground state.

In [5]:
# setup VQE 
# setup optimizer, use L_BFGS_B optimizer for example
lbfgs = L_BFGS_B(maxfun=1000, factr=10, iprint=10)

# setup variational form generator (generate trial circuits for VQE)
var_form = TwoLocal(newqubitOp_jw.num_qubits, 'ry', 'cz', reps=5, entanglement=[[0, 1], [1, 2], [2, 3]])

# setup VQE with operator, variational form, and optimizer
vqe_algorithm = VQE(newqubitOp_jw, var_form, lbfgs)

backend = BasicAer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend)

results = vqe_algorithm.run(quantum_instance)
print("Minimum value: {}".format(results.eigenvalue.real))
print("Minimum value: {}".format(results.eigenvalue.real - energy_shift))
print("Parameters: {}".format(results.optimal_point))

Minimum value: -0.02030703892349
Minimum value: -1.8572750301264738
Parameters: [ 5.28462733  3.50885712  1.19186098 -1.31257921  5.55321399 -0.49555501
 -5.10169198 -5.4784798  -0.88598656  4.05399968  3.26356323  6.05812004
  3.41375049  3.10439713 -1.10313698 -0.57661347  2.65713956 -1.90680973
  6.10125861  0.49985016  4.83574309 -2.15677133 -2.96541528 -2.28211722]
